In [ ]:
import numpy as np
import itertools
from dataclasses import dataclass
import PIL.Image
import PIL.ImageDraw
import PIL.ImageFont
import matplotlib.pyplot as plt

In [ ]:
print('\u2C00')

In [ ]:
alphabet = list( map( chr, range( ord( '\u2C00' ), ord( '\u2C2A' ) ) ) )
for s in alphabet:
    print(s)
    #Ⱀ, Ⰿ das Ⰸ

In [ ]:
def first_nonzero(arr, axis, invalid_val=-1):
    mask = arr!=0
    return np.min( np.where(mask.any(axis=axis), mask.argmax(axis=axis), 100000) )
def last_nonzero(arr, axis):
    flipped_nz = first_nonzero(np.flip(arr), axis)
    return arr.shape[axis] - flipped_nz

In [ ]:
@dataclass
class Features:
    weight: list
    weight_norm: list
    centroid: tuple
    centroid_norm: tuple
    axes_momentum: tuple
    axes_momentum_norm: tuple
    profiles: list

class FeaturesCalculator:
    @staticmethod
    def split_to_quadrants(img):
        sz = img.shape
        hz = ( img.shape[0] // 2, img.shape[1] // 2 )
        return [
            img[0:hz[0], 0:hz[1]],
            img[0:hz[0], hz[1]:sz[1]],
            img[hz[0]:sz[0], 0:hz[1]],
            img[hz[0]:sz[0], hz[1]:sz[1]]
        ]

    @staticmethod
    def weight_per_quadrant(img):
        return list(map( lambda q: np.sum(q), FeaturesCalculator.split_to_quadrants( img ) ))

    @staticmethod
    def weight_norm_per_quadrant(img):
        return list(map( lambda q: np.sum(q) / ( q.shape[0] * q.shape[1] ), FeaturesCalculator.split_to_quadrants( img ) ))

    @staticmethod
    def axes_reduce(img, fs):
        if not isinstance(fs, tuple):
            fs = (fs, fs)

        meanx = 0
        meany = 0
        for y, x in itertools.product( range( img.shape[0] ), range( img.shape[1] ) ):
            meany += fs[0]( y ) * img[y, x]
            meanx += fs[1]( x ) * img[y, x]

        s = np.sum(img)
        return meany/s, meanx/s

    @staticmethod
    def centroid(img):
        return FeaturesCalculator.axes_reduce(img, lambda x: x)

    @staticmethod
    def centroid_norm(img):
        c = FeaturesCalculator.centroid( img )
        return c[0] / img.shape[0], c[1] / img.shape[1]

    @staticmethod
    def axes_momentum(img):
        c = FeaturesCalculator.centroid(img)
        return FeaturesCalculator.axes_reduce(img, ( lambda y: (y - c[0])**2, lambda x: (x - c[1])**2 ) )

    @staticmethod
    def axes_momentum_norm(img):
        m = FeaturesCalculator.axes_momentum(img)
        return m[0] / img.shape[0], m[1] / img.shape[1]

    @staticmethod
    def proj(img, axis):
        return np.sum(img, axis)

    def __init__(self, img):
        self.features = Features(
            weight = FeaturesCalculator.weight_per_quadrant( img ),
            weight_norm = FeaturesCalculator.weight_norm_per_quadrant( img ),
            centroid = FeaturesCalculator.centroid( img ),
            centroid_norm = FeaturesCalculator.centroid_norm( img ),
            axes_momentum = FeaturesCalculator.axes_momentum( img ),
            axes_momentum_norm = FeaturesCalculator.axes_momentum_norm( img ),
            profiles=[FeaturesCalculator.proj( img, 0 ), FeaturesCalculator.proj( img, 1 )]
        )



In [ ]:
base_path = '/home/andrew/Projects/MEPhI/ImageAndAudioProcessing/data/glag_letters/'
stats_file = base_path + 'stats.csv'

with open( stats_file, 'w' ) as f:
    f.write(
        'index,weight,weight_norm,centroid,centroid_norm,axes_momentum,axes_momentum_norm\n'
    )

for i, l in enumerate( alphabet ):
    img = PIL.Image.new( '1', (200, 200), 0 )
    d = PIL.ImageDraw.Draw( img )
    d.text( (0, -20), l, 1, PIL.ImageFont.truetype( '/usr/share/fonts/noto-sans-glagolitic/NotoSansGlagolitic-Regular.ttf', 120, encoding='unic' ) )

    img_np = np.array( img )
    left = first_nonzero(img_np, 1)
    right = last_nonzero(img_np, 1)
    top = first_nonzero(img_np, 0)
    bottom = last_nonzero(img_np, 0)

    img_np = img_np[top:bottom, left:right]

    features = FeaturesCalculator( img_np ).features
    plt.plot(features.profiles[0])
    plt.savefig( base_path + f'{i}_profile_0.png' )
    plt.clf()
    plt.plot(features.profiles[1])
    plt.savefig( base_path + f'{i}_profile_1.png' )
    plt.clf()

    with open(stats_file, 'a') as f:
        f.writelines(
            f'{i},'
            f'{features.weight[0]};{features.weight[1]},'
            f'{features.weight_norm[0]};{features.weight_norm[1]},'
            f'{features.centroid[0]};{features.centroid[1]},'
            f'{features.centroid_norm[0]};{features.centroid_norm[1]},'
            f'{features.axes_momentum[0]};{features.axes_momentum[1]},'
            f'{features.axes_momentum_norm[0]};{features.axes_momentum_norm[1]}\n'
        )

    img = PIL.Image.fromarray( img_np )

    img.save( base_path + f'{i}.tif', 'TIFF' )
